# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

With the help of python notebooks provided in 1st and 2nd project of this nanodegree program ('Optimizing a pipeline in Azure' and 'Operationalizing machine learning'), I have imported following basic dependencies required to complete this project. Any other specific dependecy will be imported as we proceed further in this project.

In [26]:
import logging
import os
import csv

from matplotlib import pyplot as pyplot
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep

# CheckIng core SDK version number
print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.26.0


In [27]:
# creating an automl experiment in our workspace

# initializing a workspace
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

# choosing a name for experiment
experiment_name = 'AutoML-Exp002'
project_folder = './AutoML-pipeline-project'

# creating the experiment
experiment = Experiment(ws, experiment_name)
experiment.start_logging()
experiment

aml-02
bcp_coe_plioto
eastus2
eeb1fe4f-1f40-4db4-895a-353699752070


Name,Workspace,Report Page,Docs Page
AutoML-Exp002,aml-02,Link to Azure Machine Learning studio,Link to Documentation


In [28]:
# creating an AMLCompute cluster for running the experiment

# importing required dependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choosing a name for our CPU cluster
amlcompute_cluster_name = "CI-AutoML01"

# Verifying that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
compute_target.get_status()

Found existing cluster, use it.
Succeeded...................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview

The dataset that we will be using for this project is the [Heart Failure Prediction](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) dataset from Kaggle, it consist of 12 features.

**12 clinical features:**

* age - Age

* anaemia - Decrease of red blood cells or hemoglobin (boolean)

* creatinine_phosphokinase - Level of the CPK enzyme in the blood (mcg/L)

* diabetes - If the patient has diabetes (boolean)

* ejection_fraction - Percentage of blood leaving the heart at each contraction (percentage)

* high_blood_pressure - If the patient has hypertension (boolean)
  
* platelets - Platelets in the blood (kiloplatelets/mL)

* serum_creatinine - Level of serum creatinine in the blood (mg/dL)

* serum_sodium - Level of serum sodium in the blood (mEq/L)
  
* sex - Woman or man (binary)
  
* smoking - If the patient smokes or not (boolean)

* time - Follow-up period (days)


We will be predicting the following output:

DEATH_EVENT if the patient deceased during the follow-up period (boolean)

In [29]:
found = False
key = "Heart Failure Prediction"
description_text = "Heart Failure Prediction DataSet for Udacity Project 3"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("The dataset is loaded")

if not found:
        # Creating Dataset and register it into Workspace
        data1 = "https://raw.githubusercontent.com/RollyAngell/ML-Azure-Udacity/main/Project%203%20-%20Casptone%20Project%20AutoML%20vs%20HyperDrive/heart%20failure%20clinical%20records%20dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(data1)        
        # Registering Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

The dataset is loaded


In [30]:
dataset

{
  "source": [
    "https://raw.githubusercontent.com/RollyAngell/ML-Azure-Udacity/main/Project%203%20-%20Casptone%20Project%20AutoML%20vs%20HyperDrive/heart%20failure%20clinical%20records%20dataset.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "a7712ae5-e176-4420-98af-e1cef0c5d7c9",
    "name": "Heart Failure Prediction",
    "version": 1,
    "description": "Heart Failure Prediction DataSet for Udacity Project 3",
    "workspace": "Workspace.create(name='aml-02', subscription_id='eeb1fe4f-1f40-4db4-895a-353699752070', resource_group='bcp_coe_plioto')"
  }
}

In [31]:
df_ = dataset.to_pandas_dataframe()
df_.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [32]:
df_["DEATH_EVENT"].value_counts()

0    203
1     96
Name: DEATH_EVENT, dtype: int64

## AutoML Configuration

Explain why you chose the automl settings and cofiguration you used below.

|Setting |Why?|
|-|-|
|**experiment_timeout_minutes**|We choose 30 minutos that means the experiment will exit after that time.|
|**max_concurrent_iterations**|To get result quickly we choose 5 that means that 5 iterations at the same time.|
|**primary_metric**|This is the metric that we want to optimize (accuracy). I will use 'AUC_weighted' as 'primary_metric' parameter. AUC means the area under the Receiver Operating Characteristic Curve which plots the relationship between true positive rate and false positive rate. Since our dataset doesn't have high class imbalance, we can use ROC method for judging the performance of a model. I will use AUC_weighted in order to mitigate the effects of whatever little imbalance is there in the dataset. AUC_weighted is the arithmetic mean of the score for each class, weighted by the number of true instances in each class.|
|**task**|classification |
|**compute_target**|To define the compute cluster we will be using |
|**training_data**|It is the training dataset to be used for the experiment.|
|**label_column_name**|To specify the dependent variable that we are trying to classify. I will enter 'DEATH_EVENT' as 'label_column_name' parameter.|
|**path**|To specify the dependent variable that we are trying to classify |
|**enable_early_stopping**|we can choose to terminate the experiment if the score stops improving in the short term. I will enter 'True' as 'enable_early_stopping' parameter.|
|**featurization**|To specify the dependent variable that we are trying to classify |
|**debug_log**|To specify the dependent variable that we are trying to classify |
|**enable_onnx_compatible_models **|To specify the dependent variable that we are trying to classify |

In [33]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes" : 30,
    "max_concurrent_iterations" : 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                            task="classification",
                            training_data=dataset,
                            label_column_name="DEATH_EVENT",
                            path='./AutoML-pipeline-project',
                            enable_early_stopping=True,
                            featurization='auto',
                            enable_onnx_compatible_models=True,
                            debug_log="automl_errors.log",
                            **automl_settings
)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(config = automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on CI-AutoML01 with default configuration
Running on remote compute: CI-AutoML01


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-Exp002,AutoML_92538d93-6457-4f2f-88c6-8a44ea0fb308,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

*************************************************

In [ ]:
# waiting for completion of remote_run while showing its output
remote_run.wait_for_completion(show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
# importing required dependencies
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve best model from AutoML Run

# importing required dependencies
import joblib

best_automl_run, best_automl_fitted_model = remote_run.get_output()
print(best_automl_run)
print(best_automl_fitted_model)

In [ ]:
#TODO: Save the best model
joblib.dump(best_automl_fitted_model,'best_automl_model.pkl')

In [ ]:
# getting the details of the best model produced by automl
best_automl_run.get_tags()

In [ ]:
from pprint import pprint

def print_model(model,prefix=""):
    for step in model.steps:
        print(prefix+step[0])
        if hasattr(step[1],'estimators') and hasattr(step[1],'weights'):
            pprint({'estimators':list(e[0] for e in step[1].estimators), 'weights':step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1],estimator[0]+'-')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_automl_fitted_model)

In [ ]:
# retrieving the best model as ONNX model
best_auto_run, best_onnx_model = remote_run.get_output(return_onnx_model=True)

In [ ]:
# importing required dependencies
from azureml.automl.runtime.onnx_convert import OnnxConverter

# saving the best model as onnx_model
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(best_onnx_model, onnx_fl_path)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# registering the best automl model

description = 'heart failure predictions'
tags = None

model = remote_run.register_model(description = description, tags = tags)

print(remote_run.model_id)

# importing required dependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

# loading a curated environment from workspace

env = Environment.get(ws, "AzureML-AutoML")

# specifying scikit-learn as dependency
for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

# creating an inference config
inference_config = InferenceConfig(entry_script='entry_script.py', environment=env)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights = True)

# naming the service to be deployed
aci_service_name = 'automl-heart-failure-predictions'
print(aci_service_name)

# deploying the model
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# importing required dependencies
from numpy import array
import json
import requests

# creating a test sample
data = {"data": [{"age":60.000000,"anaemia":0.000000,"creatinine_phosphokinase":250.000000,"diabetes":0.000000,"ejection_fraction":38.000000,"high_blood_pressure":0.000000,"platelets":262000.000000,"serum_creatinine":1.10000,"serum_sodium":137.000000,"sex":1.000000,"smoking":0.00000,"time":115.000000}]}
td = json.dumps(data)
headers = {'Content-Type': 'application/json'}

# sending request to test the deployed webservice
resp = requests.post(aci_service.scoring_uri, td, headers=headers)
print(resp.json())
y_pred = (json.loads(resp.text))
print(y_pred)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# printing the logs of deployed web service
dep_logs = aci_service.get_logs()
for l in dep_logs.split('\n'):
    print(l)

In [ ]:
compute_target

In [ ]:
aci_service

In [ ]:
# deleting a web service
aci_service.delete()

In [ ]:
# delete compute cluster
compute_target.delete()